# Parkinson's Machine Learning Classification

## Table of Contents
1. [Initial Data Analysis](#Initial-Data-Analysis)
1. [Initial Model Benchmarks](#Initial-Model-Benchmarks)
    1. [Benchmark Results __(click here for quick summary)__](#Results)

## Initial Data Analysis 
[See Data Exploration Notebook](Data Exploration/Data_Exploration.ipynb)

The data set obviously has a class imbalance (Control 32%, Parkinsons 53%, MSA 7%, PSP 7%).  We will need to resample the data or use an algorithm like Decision Trees that are not influenced as heavily by class imbalance.  We should also take care to see how well the under-represented classes are being predicted when evaluating accuracy of our models.

Feature appear relatively normally-distributed between groups.  Nothing particularly interesting jumping out by looking at covariance and pearson correlations

## Initial Model Benchmarks
[See Modeling Benchmark Notebook](Model Benchmarks/ModelingBenchmarks.ipynb)

8 initial models were tested for __multiclass classification__ (I would ideally like to avoid composing results of multiple binary classifiers since the cost of that composition is great):
* Logistic Regression (*log*)
* SVC with Linear Kernel (*svc_lin*)
* SVC with RBF Kernel (*svc_rbf*)
* k-Nearest Neighbors (*knn*)
* Decision Trees (*rand_for*)
* Artificial Neural Networks (*ann*)
* Naive Bayes (*gnb*)
* AdaBoost (*ada*)

These models were run with 5-fold (stratified) cross validation on 80% of the training data (20% kept for holdout validation).  To address the problem of class imbalance, the data was resampled (upsampling of the minority classes) to ensure that the minority classes were well represented in the models.  Alternative imbalanced class corrections can be explored in the future.   No additional preprocessing of the data was performed (scaling, feature selection, etc..).

For most models, default hyperparameters were used (with minor tweaking) based on experience.

The "Outside Validation" data set (who sites were not included in the training data at all), was tested to measure generalizability to unseen data.

The models were evaluated by several means with particular attention paid to the performance on the minority classes:
* Cross Validation Score (Mean Accuracy of the folds)
* Holdout Data accuracy, precision (PPV), and recall (sensitivity)
* "Outside Validation" Data accuracy, precision (PPV), and recall (sensitivity)

### Results

!['test'](benchmark_accuracies.png)

k nearest neighbors, non-linear support vector, random forests, and logorithmic regression yielded cross-validation and holdout accuracies > 80%.

Each of those classifiers yield "Outside Validation" accuracies significantly less than 80%.  However, the other classifiers behave consistently between cross-validation, holdout, and validation data sets.

Of course, significant improvements are expected when we begin to introduce various preprocessing methods.

Further useful insight into the how well these models are performing on the individual class level can be seen by viewing the confusion matrices in 
[the Modeling Benchmark Notebook](Model Benchmarks/ModelingBenchmarks.ipynb)
